Ёбаные 0.8227 / 0.8233

In [1]:
import polars as pl
import pandas as pd
import numpy as np
import catboost
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
import lightgbm as lgb
import os
import optuna
from datetime import date, timedelta
from my_utils import embedding_plotter
from sklearn.preprocessing import StandardScaler
import umap.umap_ as umap
import shap
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cluster import KMeans
import hdbscan
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import faiss

In [2]:
test_start_date = date(2024, 8, 1)
val_start_date = date(2024, 7, 1)
val_end_date = date(2024, 7, 31)
train_end_date = date(2024, 6, 30)
data_path = './contest_1_data'

# Read data

In [3]:
actions_history = pl.read_parquet(os.path.join(data_path, 'actions_history'))
search_history = pl.read_parquet(os.path.join(data_path, 'search_history'))
product_information = pl.read_csv(os.path.join(data_path, 'product_information.csv'))

product_information = product_information.with_columns(
    ((pl.col("price") - pl.col("discount_price")) / pl.col("price")).alias(
        "relative_discount"
    )
)

In [4]:
pl.read_csv(os.path.join(data_path, 'action_type_info.csv'))

action_type,action_type_id
str,i64
"""click""",1
"""favorite""",2
"""order""",3
"""search""",4
"""to_cart""",5
"""view""",6


In [5]:
val_target = (
    actions_history
    .filter(pl.col('timestamp').dt.date() >= val_start_date)
    .filter(pl.col('timestamp').dt.date() <= val_end_date)
    .select('user_id', (pl.col('action_type_id') == 3).alias('has_order'))
    .group_by('user_id')
    .agg(pl.max('has_order').cast(pl.Int32).alias('target'))
)

In [6]:
val_target.group_by('target').agg(pl.count('user_id'))

target,user_id
i32,u32
0,1227381
1,647575


# Simple pipeline

## Feats

In [7]:
actions_aggs = {}
actions_id_to_suf = {
    1: "click",
    2: "favorite",
    3: "order",
    5: "to_cart",
}
for id_, suf in tqdm(actions_id_to_suf.items()):
    aggs = (
        actions_history
        .filter(pl.col('timestamp').dt.date() <= train_end_date)
        .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=30 * 4))
        .filter(pl.col('action_type_id') == id_)
        .join(
            product_information
            .select('product_id', 'discount_price', 'relative_discount'),
            on='product_id',
        )
        .group_by('user_id')
        .agg(
            pl.count('product_id').cast(pl.Int32).alias(f'num_products_{suf}'),
            pl.sum('discount_price').cast(pl.Float32).alias(f'sum_discount_price_{suf}'),
            pl.max('discount_price').cast(pl.Float32).alias(f'max_discount_price_{suf}'),
            pl.mean('relative_discount').cast(pl.Float32).alias(f'mean_relative_discount_{suf}'), # new
            pl.max('relative_discount').cast(pl.Float32).alias(f'max_relative_discount_{suf}'), # new
            pl.max('timestamp').alias(f'last_{suf}_time'),
            pl.min('timestamp').alias(f'first_{suf}_time'),
        )
        .with_columns([
            (pl.lit(val_start_date) - pl.col(f'last_{suf}_time'))
            .dt.total_days()
            .cast(pl.Int32)
            .alias(f'days_since_last_{suf}'),
            
            (pl.lit(val_start_date) - pl.col(f'first_{suf}_time'))
            .dt.total_days()
            .cast(pl.Int32)
            .alias(f'days_since_first_{suf}'),
        ])
        .select(
            'user_id',
            f'num_products_{suf}',
            f'sum_discount_price_{suf}',
            f'max_discount_price_{suf}',
            f'mean_relative_discount_{suf}',
            f'max_relative_discount_{suf}',
            f'days_since_last_{suf}',
            f'days_since_first_{suf}',
        )
    )
    actions_aggs[id_] = aggs

  0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
# search_aggs
id_ = 4
suf = 'search'
actions_aggs[id_] = (
    search_history
    .filter(pl.col('action_type_id') == id_)
    .filter(pl.col('timestamp').dt.date() <= train_end_date)
    .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=30 * 4))
    .group_by('user_id')
    .agg(
        pl.count('search_query').cast(pl.Int32).alias(f'num_{suf}'),
        pl.max('timestamp').alias(f'last_{suf}_time'),
        pl.min('timestamp').alias(f'first_{suf}_time'),
    )
    .with_columns([
        (pl.lit(val_start_date) - pl.col(f'last_{suf}_time'))
        .dt.total_days()
        .cast(pl.Int32)
        .alias(f'days_since_last_{suf}'),

        (pl.lit(val_start_date) - pl.col(f'first_{suf}_time'))
        .dt.total_days()
        .cast(pl.Int32)
        .alias(f'days_since_first_{suf}'),
    ])
    .select(
        'user_id',
        f'num_{suf}',
        f'days_since_last_{suf}',
        f'days_since_first_{suf}',
    )
)

In [9]:
actions_aggs.keys()

dict_keys([1, 2, 3, 5, 4])

In [10]:
df = val_target
for _, actions_aggs_df in actions_aggs.items():
    df = (
        df
        .join(actions_aggs_df, on='user_id', how='left')
    )

In [11]:
df

user_id,target,num_products_click,sum_discount_price_click,max_discount_price_click,mean_relative_discount_click,max_relative_discount_click,days_since_last_click,days_since_first_click,num_products_favorite,sum_discount_price_favorite,max_discount_price_favorite,mean_relative_discount_favorite,max_relative_discount_favorite,days_since_last_favorite,days_since_first_favorite,num_products_order,sum_discount_price_order,max_discount_price_order,mean_relative_discount_order,max_relative_discount_order,days_since_last_order,days_since_first_order,num_products_to_cart,sum_discount_price_to_cart,max_discount_price_to_cart,mean_relative_discount_to_cart,max_relative_discount_to_cart,days_since_last_to_cart,days_since_first_to_cart,num_search,days_since_last_search,days_since_first_search
i32,i32,i32,f32,f32,f32,f32,i32,i32,i32,f32,f32,f32,f32,i32,i32,i32,f32,f32,f32,f32,i32,i32,i32,f32,f32,f32,f32,i32,i32,i32,i32,i32
1384102,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
9628369,0,1,426.0,426.0,0.492857,0.492857,57,57,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,426.0,426.0,0.492857,0.492857,57,57,3,57,57
2819812,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,2,2
1766085,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1658950,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
7194562,0,2,353.0,182.0,0.108968,0.212121,29,29,null,null,null,null,null,null,null,1,179.0,179.0,0.0,0.0,29,29,1,179.0,179.0,0.0,0.0,29,29,null,null,null
4974206,1,35,24925.0,1945.0,0.371692,0.707991,12,118,null,null,null,null,null,null,null,18,5029.0,1211.0,0.203663,0.592119,2,25,69,25297.0,1436.0,0.204439,0.703869,2,120,78,2,120
6933305,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,188.0,188.0,0.064677,0.064677,64,64,null,null,null


### Конверсии

Добавляю числовые признаки, имеющие смысл конверсий. Взял конверсии связанные со всеми активностями в логическом порядке (`conversion_list` ниже). Стоит также заметить, что это не чистые конверсии в том смысле, что из-за того, что данные могут отсутствовать, или вообще путь клиента будет неравномерным, так называемая конверсия может быть больше 1 (например, если клиент сделал заказы, не добавляя товар в избранное и т.д.).

In [12]:
conversion_list = [
    ('click', 'favorite'),
    ('click', 'to_cart'),
    ('click', 'order'),
    ('favorite', 'to_cart'),
    ('favorite', 'order'),
    ('to_cart', 'order')
]
df_conversions = {}
# конверсия из A в B
for A, B in conversion_list:
    df_conversions[f"{A}_to_{B}"] = (
        df.filter((pl.col(f"num_products_{A}") > 0) & (pl.col(f"num_products_{B}") > 0))
        .with_columns(
            (pl.col(f"num_products_{B}") / pl.col(f"num_products_{A}")).alias(
                f"{A}_to_{B}"
            ),
        ).select(["user_id", f"{A}_to_{B}"])
    )
for A, B in conversion_list:
    df = df.join(df_conversions[f"{A}_to_{B}"], on="user_id", how="left")
df_conversions

{'click_to_favorite': shape: (289_869, 2)
 ┌──────────┬───────────────────┐
 │ user_id  ┆ click_to_favorite │
 │ ---      ┆ ---               │
 │ i32      ┆ f64               │
 ╞══════════╪═══════════════════╡
 │ 10334257 ┆ 0.006757          │
 │ 6447317  ┆ 0.005348          │
 │ 2811163  ┆ 0.010101          │
 │ 3197704  ┆ 0.016393          │
 │ 425913   ┆ 0.2               │
 │ …        ┆ …                 │
 │ 6781389  ┆ 0.05              │
 │ 5797262  ┆ 2.666667          │
 │ 4728666  ┆ 0.09375           │
 │ 1718317  ┆ 8.75              │
 │ 10456724 ┆ 0.090909          │
 └──────────┴───────────────────┘,
 'click_to_to_cart': shape: (867_406, 2)
 ┌──────────┬──────────────────┐
 │ user_id  ┆ click_to_to_cart │
 │ ---      ┆ ---              │
 │ i32      ┆ f64              │
 ╞══════════╪══════════════════╡
 │ 9628369  ┆ 1.0              │
 │ 1842988  ┆ 0.222222         │
 │ 10432266 ┆ 5.0              │
 │ 10334257 ┆ 0.918919         │
 │ 712301   ┆ 1.142857         │
 │ …    

### Окно по 7, 14, 30, 60 дней

Добавляю аналоги агрегационных признаков, сгенерированных семинаристом, при том изменении, что они берутся за последние 7/14/30 дней. Эти значения могут отображать, насколько активен клиент на относительно коротком промежутке времени.

In [13]:
action_aggs_window = {}
actions_id_to_suf = {
        1: "click",
        2: "favorite",
        3: "order",
        5: "to_cart",
    }
for window in [7, 14, 30, 60]:
    action_aggs_window[window] = {}
    for id_, suf in tqdm(actions_id_to_suf.items()):
        action_aggs_window[window][id_] = (
            actions_history
            .filter(pl.col('timestamp').dt.date() <= train_end_date)
            .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=window))
            .filter(pl.col('action_type_id') == id_)
            .join(
                product_information
                .select('product_id', 'discount_price', 'relative_discount'),
                on='product_id',
            )
            .group_by('user_id')
            .agg(
                pl.count('product_id').cast(pl.Int32).alias(f'num_products_{suf}_{window}'),
                pl.sum('discount_price').cast(pl.Float32).alias(f'sum_discount_price_{suf}_{window}'),
                pl.max('discount_price').cast(pl.Float32).alias(f'max_discount_price_{suf}_{window}'),
                pl.mean('relative_discount').cast(pl.Float32).alias(f'mean_relative_discount_{suf}_{window}'),
                pl.max('relative_discount').cast(pl.Float32).alias(f'max_relative_discount_{suf}_{window}')
            )
            .select(
                'user_id',
                f'num_products_{suf}_{window}',
                f'sum_discount_price_{suf}_{window}',
                f'max_discount_price_{suf}_{window}',
                f'mean_relative_discount_{suf}_{window}',
                f'max_relative_discount_{suf}_{window}',
            )
        )
id_ = 4
suf = 'search'
for window in [7, 14, 30, 60]:
    action_aggs_window[window][id_] = (
        search_history
        .filter(pl.col('action_type_id') == id_)
        .filter(pl.col('timestamp').dt.date() <= train_end_date)
        .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=window))
        .group_by('user_id')
        .agg(
            pl.count('search_query').cast(pl.Int32).alias(f'num_{suf}_{window}'),
        )
        .select(
            'user_id',
            f'num_{suf}_{window}',
        )
    )
for window in [7, 14, 30, 60]:
    for action_id in range(1, 6):
        df = df.join(action_aggs_window[window][action_id], on="user_id", how="left")
action_aggs_window

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

{7: {1: shape: (538_744, 6)
  ┌─────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬────────────────────┐
  │ user_id ┆ num_products_click_ ┆ sum_discount_price_ ┆ max_discount_price_ ┆ mean_relative_disco ┆ max_relative_disco │
  │ ---     ┆ 7                   ┆ click_7             ┆ click_7             ┆ unt_click_7         ┆ unt_click_7        │
  │ i32     ┆ ---                 ┆ ---                 ┆ ---                 ┆ ---                 ┆ ---                │
  │         ┆ i32                 ┆ f32                 ┆ f32                 ┆ f32                 ┆ f32                │
  ╞═════════╪═════════════════════╪═════════════════════╪═════════════════════╪═════════════════════╪════════════════════╡
  │ 6704757 ┆ 12                  ┆ 11091.0             ┆ 3924.0              ┆ 0.313295            ┆ 0.789546           │
  │ 2279777 ┆ 2                   ┆ 582.0               ┆ 463.0               ┆ 0.03876             ┆ 0.077519 

### Средняя длина поискового запроса

In [14]:
df = df.join(
    search_history
    .filter(pl.col('timestamp').dt.date() <= train_end_date)
    .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=window))
    .group_by("user_id").agg(
        pl.col("search_query").len().alias("mean_length_of_query")
    ).cast(pl.Int32),
    on="user_id", how="left"
)

### Число активностей по виджетам

In [15]:
df = df.join(
    (
        actions_history
        .filter(pl.col('timestamp').dt.date() <= train_end_date)
        .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=30 * 4))
        .pivot(
            index="user_id",          # группируем по user_id
            on="widget_name_id",      # создаём столбцы для каждого widget_id
            values="widget_name_id",       # будем агрегировать по widget_id
            aggregate_function="len"      # считаем количество записей
        )
        .fill_null(0)
        .select(
            ["user_id"] + sorted(list(actions_history['widget_name_id'].unique().cast(pl.String)))
        )
        .drop("13")
        .drop("3")
    ), on="user_id", how="left"
)

df = df.join(
    (
        search_history
        .filter(pl.col('timestamp').dt.date() <= train_end_date)
        .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=30 * 4))
        .pivot(
            index="user_id",          # группируем по user_id
            on="widget_name_id",      # создаём столбцы для каждого widget_id
            values="widget_name_id",       # будем агрегировать по widget_id
            aggregate_function="len"      # считаем количество записей
        )
        .fill_null(0)
        .select(
            ["user_id"] + sorted(list(search_history['widget_name_id'].unique().cast(pl.String)))
        )
    ), on="user_id", how="left"
)

## Categorical features

#### Был ли заказ за последний месяц, два месяца, три месяца

In [16]:
df = df.join(
    (
        actions_history
        .filter(pl.col('timestamp').dt.date() <= train_end_date)
        .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=30 * 4))
        .filter(pl.col('action_type_id') == 3).group_by("user_id")
        .agg(
            pl.col("timestamp").max().alias("last_order_ts"),
            purchased_last_month=pl.col("timestamp")
                .filter(pl.col("timestamp") >= train_end_date - timedelta(days=30))
                .count() > 0,
            purchased_last_2months=pl.col("timestamp")
                .filter(pl.col("timestamp") >= train_end_date - timedelta(days=30 * 2))
                .count() > 0,
            purchased_last_3months=pl.col("timestamp")
                .filter(pl.col("timestamp") >= train_end_date - timedelta(days=30 * 3))
                .count() > 0
        )
    ).drop("last_order_ts"),
    on="user_id", how="left"
)
df = df.with_columns(pl.col("purchased_last_month").fill_null(False))
df = df.with_columns(pl.col("purchased_last_2months").fill_null(False))
df = df.with_columns(pl.col("purchased_last_3months").fill_null(False))

#### Было ли добавление в корзину за последний месяц, два месяца, три месяца

In [17]:
df = df.join(
    (
        actions_history
        .filter(pl.col('timestamp').dt.date() <= train_end_date)
        .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=30 * 4))
        .filter(pl.col('action_type_id') == 5).group_by("user_id")
        .agg(
            pl.col("timestamp").max().alias("last_to_cart_ts"),
            to_cart_last_month=pl.col("timestamp")
                .filter(pl.col("timestamp") >= train_end_date - timedelta(days=30))
                .count() > 0,
            to_cart_last_2months=pl.col("timestamp")
                .filter(pl.col("timestamp") >= train_end_date - timedelta(days=30 * 2))
                .count() > 0,
            to_cart_last_3months=pl.col("timestamp")
                .filter(pl.col("timestamp") >= train_end_date - timedelta(days=30 * 3))
                .count() > 0
        )
    ).drop("last_to_cart_ts"),
    on="user_id", how="left"
)

df = df.with_columns(pl.col("to_cart_last_month").fill_null(False))
df = df.with_columns(pl.col("to_cart_last_2months").fill_null(False))
df = df.with_columns(pl.col("to_cart_last_3months").fill_null(False))

### Топ-k категорий товаров на виды активности

In [18]:
k = 3
actions_id_to_suf = {
    1: "click",
    2: "favorite",
    3: "order",
    5: "to_cart",
}
for id_, suf in tqdm(actions_id_to_suf.items()):
    df = df.join(
        (
            actions_history
            .filter(pl.col('timestamp').dt.date() <= train_end_date)
            .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=30 * 4))
            .filter(pl.col("action_type_id") == id_)
            .join(
                product_information.select(["product_id", "category_name"]),
                on="product_id",
            )
            .group_by("user_id", "category_name")
            .agg(pl.len().alias("count"))
            .with_columns(
                pl.col("category_name").sort_by(by=["count", "category_name"], descending=[True, False])
                .over("user_id")
                .alias("sorted_values")
            )
            .group_by("user_id")
            .agg(pl.col("sorted_values").head(k).alias("top_values"))
            .with_columns(
                pl.col("top_values").list.to_struct(fields=[f"top{i}_cat_in_{suf}" for i in range(1, k + 1)])
            )
            .unnest(columns="top_values")
        ),
        on='user_id', how='left'
    )
    
for col in [col for col in df.columns if col.find("cat") != -1]:
    df = df.with_columns(pl.col(col).fill_null(""))

  0%|          | 0/4 [00:00<?, ?it/s]

### Топ-k типов виджетов на виды активности

In [19]:
actions_id_to_suf = {
    1: "click",
    2: "favorite",
    # 3: "order", так как в заказе только один виджет, нет смысла
    5: "to_cart",
}
for id_, suf in tqdm(actions_id_to_suf.items()):
    df = df.join(
        (
            actions_history
            .filter(pl.col('timestamp').dt.date() <= train_end_date)
            .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=30 * 4))
            .filter(pl.col("action_type_id") == id_)
            .group_by("user_id", "widget_name_id")
            .agg(pl.len().alias("count"))
            .with_columns(
                pl.col("widget_name_id").sort_by(by=["count", "widget_name_id"], descending=[True, False])
                .over("user_id")
                .alias("sorted_values")
            )
            .group_by("user_id")
            .agg(pl.col("sorted_values").head(k).alias("top_values"))
            .with_columns(
                pl.col("top_values").list.to_struct(fields=[f"top{i}_widget_in_{suf}" for i in range(1, k + 1)])
            )
            .unnest(columns="top_values")
        ),
        on='user_id', how='left'
    )

df = df.join(
        (
        search_history
        .filter(pl.col('timestamp').dt.date() <= train_end_date)
        .filter(pl.col('timestamp').dt.date() >= train_end_date - timedelta(days=30 * 4))
        .filter(pl.col("action_type_id") == 4)
        .group_by("user_id", "widget_name_id")
        .agg(pl.len().alias("count"))
        .with_columns(
            pl.col("widget_name_id").sort_by(by=["count", "widget_name_id"], descending=[True, False])
            .over("user_id")
            .alias("sorted_values")
        )
        .group_by("user_id")
        .agg(pl.col("sorted_values").head(k).alias("top_values"))
        .with_columns(
            pl.col("top_values").list.to_struct(fields=[f"top{i}_widget_in_search" for i in range(1, k + 1)])
        ).unnest(columns="top_values")
    ),
    on='user_id', how='left'
)

for col in [col for col in df.columns if col.find("widget") != -1]:
    df = df.with_columns(pl.col(col).fill_null(0))

  0%|          | 0/3 [00:00<?, ?it/s]

## Обучение модели

In [20]:
df_pd = df.to_pandas()

In [21]:
mask = df_pd.user_id % 10 <= 6

In [22]:
df_pd.columns

Index(['user_id', 'target', 'num_products_click', 'sum_discount_price_click',
       'max_discount_price_click', 'mean_relative_discount_click',
       'max_relative_discount_click', 'days_since_last_click',
       'days_since_first_click', 'num_products_favorite',
       ...
       'top3_widget_in_click', 'top1_widget_in_favorite',
       'top2_widget_in_favorite', 'top3_widget_in_favorite',
       'top1_widget_in_to_cart', 'top2_widget_in_to_cart',
       'top3_widget_in_to_cart', 'top1_widget_in_search',
       'top2_widget_in_search', 'top3_widget_in_search'],
      dtype='object', length=178)

In [23]:
actions_id_to_suf = {
    1: "click",
    2: "favorite",
    3: "order",
    4: "search",
    5: "to_cart",
}
cols = [col for col in df_pd.columns if col not in ['user_id', 'target']]
cat_features = (
    [
        "purchased_last_month",
        "purchased_last_2months",
        "purchased_last_3months",
        "to_cart_last_month",
        "to_cart_last_2months",
        "to_cart_last_3months",
    ]
    + [
        f"top{i}_cat_in_{actions_id_to_suf[id_]}"
        for i in range(1, k + 1)
        for id_ in [1, 2, 3, 5]
    ]
    + [
        f"top{i}_widget_in_{actions_id_to_suf[id_]}"
        for i in range(1, k + 1)
        for id_ in [1, 2, 4, 5]
    ]
)

#### CatBoost

In [ ]:
train_pool = catboost.Pool(
    df_pd.loc[mask, cols],
    label=df_pd.loc[mask].target,
    cat_features=cat_features
)
eval_pool = catboost.Pool(
    df_pd.loc[~mask, cols],
    label=df_pd.loc[~mask].target,
    cat_features=cat_features
)

In [ ]:
train_pool.shape, eval_pool.shape

In [ ]:
params = {
    'iterations': 200,
    'depth': 7, 
    'learning_rate': 0.1, 
    'random_state': 1,
    'eval_metric': 'AUC',
    'loss_function': 'Logloss',
    'task_type': 'GPU',
}

In [ ]:
model = catboost.CatBoost(params)
model.fit(
    train_pool, 
    eval_set=eval_pool,
    use_best_model=True,
    verbose=10,
    early_stopping_rounds=50,
)

In [ ]:
fi = model.get_feature_importance(eval_pool, prettified=True)
fi.head(50)

#### LighGBM

# Prediction on Test

In [28]:
test_users_submission = (
    pl.read_csv(os.path.join(data_path, 'test_users.csv'))
)

In [29]:
actions_aggs = {}
actions_id_to_suf = {
    1: "click",
    2: "favorite",
    3: "order",
    5: "to_cart",
}
for id_, suf in tqdm(actions_id_to_suf.items()):
    aggs = (
        actions_history
        .filter(pl.col('timestamp').dt.date() <= val_end_date)
        .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=30 * 4))
        .filter(pl.col('action_type_id') == id_)
        .join(
            product_information
            .select('product_id', 'discount_price', 'relative_discount'),
            on='product_id',
        )
        .group_by('user_id')
        .agg(
            pl.count('product_id').cast(pl.Int32).alias(f'num_products_{suf}'),
            pl.sum('discount_price').cast(pl.Float32).alias(f'sum_discount_price_{suf}'),
            pl.max('discount_price').cast(pl.Float32).alias(f'max_discount_price_{suf}'),
            pl.mean('relative_discount').cast(pl.Float32).alias(f'mean_relative_discount_{suf}'), # new
            pl.max('relative_discount').cast(pl.Float32).alias(f'max_relative_discount_{suf}'), # new
            pl.max('timestamp').alias(f'last_{suf}_time'),
            pl.min('timestamp').alias(f'first_{suf}_time'),
        )
        .with_columns([
            (pl.lit(val_start_date) - pl.col(f'last_{suf}_time'))
            .dt.total_days()
            .cast(pl.Int32)
            .alias(f'days_since_last_{suf}'),
            
            (pl.lit(val_start_date) - pl.col(f'first_{suf}_time'))
            .dt.total_days()
            .cast(pl.Int32)
            .alias(f'days_since_first_{suf}'),
        ])
        .select(
            'user_id',
            f'num_products_{suf}',
            f'sum_discount_price_{suf}',
            f'max_discount_price_{suf}',
            f'mean_relative_discount_{suf}',
            f'max_relative_discount_{suf}',
            f'days_since_last_{suf}',
            f'days_since_first_{suf}',
        )
    )
    actions_aggs[id_] = aggs

  0%|          | 0/4 [00:00<?, ?it/s]

In [30]:
# search_aggs
id_ = 4
suf = 'search'
actions_aggs[id_] = (
    search_history
    .filter(pl.col('action_type_id') == id_)
    .filter(pl.col('timestamp').dt.date() <= val_end_date)
    .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=30 * 4))
    .group_by('user_id')
    .agg(
        pl.count('search_query').cast(pl.Int32).alias(f'num_{suf}'),
        pl.max('timestamp').alias(f'last_{suf}_time'),
        pl.min('timestamp').alias(f'first_{suf}_time'),
    )
    .with_columns([
        (pl.lit(test_start_date) - pl.col(f'last_{suf}_time'))
        .dt.total_days()
        .cast(pl.Int32)
        .alias(f'days_since_last_{suf}'),

        (pl.lit(test_start_date) - pl.col(f'first_{suf}_time'))
        .dt.total_days()
        .cast(pl.Int32)
        .alias(f'days_since_first_{suf}'),
    ])
    .select(
        'user_id',
        f'num_{suf}',
        f'days_since_last_{suf}',
        f'days_since_first_{suf}',
    )
)

In [31]:
df = test_users_submission
for _, actions_aggs_df in actions_aggs.items():
    df = (
        df
        .join(actions_aggs_df, on='user_id', how='left')
    )

### Конверсии

In [32]:
conversion_list = [
    ('click', 'favorite'),
    ('click', 'to_cart'),
    ('click', 'order'),
    ('favorite', 'to_cart'),
    ('favorite', 'order'),
    ('to_cart', 'order')
]
df_conversions = {}
# конверсия из A в B
for A, B in conversion_list:
    df_conversions[f"{A}_to_{B}"] = (
        df.filter((pl.col(f"num_products_{A}") > 0) & (pl.col(f"num_products_{B}") > 0))
        .with_columns(
            (pl.col(f"num_products_{B}") / pl.col(f"num_products_{A}")).alias(
                f"{A}_to_{B}"
            ),
        ).select(["user_id", f"{A}_to_{B}"])
    )
for A, B in conversion_list:
    df = df.join(df_conversions[f"{A}_to_{B}"], on="user_id", how="left")

### Окна по 7 / 14 / 30 / 60 дней

In [33]:
action_aggs_window = {}
actions_id_to_suf = {
        1: "click",
        2: "favorite",
        3: "order",
        5: "to_cart",
    }
for window in [7, 14, 30, 60]:
    action_aggs_window[window] = {}
    for id_, suf in tqdm(actions_id_to_suf.items()):
        action_aggs_window[window][id_] = (
            actions_history
            .filter(pl.col('timestamp').dt.date() <= val_end_date)
            .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=window))
            .filter(pl.col('action_type_id') == id_)
            .join(
                product_information
                .select('product_id', 'discount_price', 'relative_discount'),
                on='product_id',
            )
            .group_by('user_id')
            .agg(
                pl.count('product_id').cast(pl.Int32).alias(f'num_products_{suf}_{window}'),
                pl.sum('discount_price').cast(pl.Float32).alias(f'sum_discount_price_{suf}_{window}'),
                pl.max('discount_price').cast(pl.Float32).alias(f'max_discount_price_{suf}_{window}'),
                pl.mean('relative_discount').cast(pl.Float32).alias(f'mean_relative_discount_{suf}_{window}'),
                pl.max('relative_discount').cast(pl.Float32).alias(f'max_relative_discount_{suf}_{window}')
            )
            .select(
                'user_id',
                f'num_products_{suf}_{window}',
                f'sum_discount_price_{suf}_{window}',
                f'max_discount_price_{suf}_{window}',
                f'mean_relative_discount_{suf}_{window}',
                f'max_relative_discount_{suf}_{window}',
            )
        )
id_ = 4
suf = 'search'
for window in [7, 14, 30, 60]:
    action_aggs_window[window][id_] = (
        search_history
        .filter(pl.col('action_type_id') == id_)
        .filter(pl.col('timestamp').dt.date() <= val_end_date)
        .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=window))
        .group_by('user_id')
        .agg(
            pl.count('search_query').cast(pl.Int32).alias(f'num_{suf}_{window}'),
        )
        .select(
            'user_id',
            f'num_{suf}_{window}',
        )
    )
for window in [7, 14, 30, 60]:
    for action_id in range(1, 6):
        df = df.join(action_aggs_window[window][action_id], on="user_id", how="left")

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

### Средняя длина поискового запроса

In [34]:
df = df.join(
    search_history
    .filter(pl.col('timestamp').dt.date() <= val_end_date)
    .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=window))
    .group_by("user_id").agg(
        pl.col("search_query").len().alias("mean_length_of_query")
    ).cast(pl.Int32),
    on="user_id", how="left"
)

### Число активностей по виджетам

In [35]:
widget_actions_list = sorted(list((
                actions_history
                .filter(pl.col('timestamp').dt.date() <= val_end_date)
                .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=30 * 4))
            )['widget_name_id'].unique().cast(pl.String)))
widget_actions_list = [col for col in widget_actions_list if col not in ['3', '13']]

df = df.join(
    (
        actions_history
        .filter(pl.col('timestamp').dt.date() <= val_end_date)
        .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=30 * 4))
        .pivot(
            index="user_id",          # группируем по user_id
            on="widget_name_id",      # создаём столбцы для каждого widget_id
            values="widget_name_id",       # будем агрегировать по widget_id
            aggregate_function="len"      # считаем количество записей
        )
        .fill_null(0)
        .select(
            ["user_id"] + widget_actions_list
        )
    ), on="user_id", how="left"
)

df = df.join(
    (
        search_history
        .filter(pl.col('timestamp').dt.date() <= val_end_date)
        .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=30 * 4))
        .pivot(
            index="user_id",          # группируем по user_id
            on="widget_name_id",      # создаём столбцы для каждого widget_id
            values="widget_name_id",       # будем агрегировать по widget_id
            aggregate_function="len"      # считаем количество записей
        )
        .fill_null(0)
        .select(
            ["user_id"] + sorted(list((
                search_history
                .filter(pl.col('timestamp').dt.date() <= val_end_date)
                .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=30 * 4))
            )['widget_name_id'].unique().cast(pl.String)))
        )
    ), on="user_id", how="left"
)

## Categorical features

#### Был ли заказ за последний месяц, два месяца, три месяца

In [36]:
df = df.join(
    (
        actions_history
        .filter(pl.col('timestamp').dt.date() <= val_end_date)
        .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=30 * 4))
        .filter(pl.col('action_type_id') == 3).group_by("user_id")
        .agg(
            pl.col("timestamp").max().alias("last_order_ts"),
            purchased_last_month=pl.col("timestamp")
                .filter(pl.col("timestamp") >= val_end_date - timedelta(days=30))
                .count() > 0,
            purchased_last_2months=pl.col("timestamp")
                .filter(pl.col("timestamp") >= val_end_date - timedelta(days=30 * 2))
                .count() > 0,
            purchased_last_3months=pl.col("timestamp")
                .filter(pl.col("timestamp") >= val_end_date - timedelta(days=30 * 3))
                .count() > 0
        )
    ).drop("last_order_ts"),
    on="user_id", how="left"
)
df = df.with_columns(pl.col("purchased_last_month").fill_null(False))
df = df.with_columns(pl.col("purchased_last_2months").fill_null(False))
df = df.with_columns(pl.col("purchased_last_3months").fill_null(False))

#### Было ли добавление в корзину за последний месяц, два месяца, три месяца

In [37]:
df = df.join(
    (
        actions_history
        .filter(pl.col('timestamp').dt.date() <= val_end_date)
        .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=30 * 4))
        .filter(pl.col('action_type_id') == 5).group_by("user_id")
        .agg(
            pl.col("timestamp").max().alias("last_to_cart_ts"),
            to_cart_last_month=pl.col("timestamp")
                .filter(pl.col("timestamp") >= val_end_date - timedelta(days=30))
                .count() > 0,
            to_cart_last_2months=pl.col("timestamp")
                .filter(pl.col("timestamp") >= val_end_date - timedelta(days=30 * 2))
                .count() > 0,
            to_cart_last_3months=pl.col("timestamp")
                .filter(pl.col("timestamp") >= val_end_date - timedelta(days=30 * 3))
                .count() > 0
        )
    ).drop("last_to_cart_ts"),
    on="user_id", how="left"
)

df = df.with_columns(pl.col("to_cart_last_month").fill_null(False))
df = df.with_columns(pl.col("to_cart_last_2months").fill_null(False))
df = df.with_columns(pl.col("to_cart_last_3months").fill_null(False))

### Топ-k категорий товаров на виды активности

In [38]:
k = 3
actions_id_to_suf = {
    1: "click",
    2: "favorite",
    3: "order",
    5: "to_cart",
}
for id_, suf in tqdm(actions_id_to_suf.items()):
    df = df.join(
        (
            actions_history
            .filter(pl.col('timestamp').dt.date() <= val_end_date)
            .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=30 * 4))
            .filter(pl.col("action_type_id") == id_)
            .join(
                product_information.select(["product_id", "category_name"]),
                on="product_id",
            )
            .group_by("user_id", "category_name")
            .agg(pl.len().alias("count"))
            .with_columns(
                pl.col("category_name").sort_by(by=["count", "category_name"], descending=[True, False])
                .over("user_id")
                .alias("sorted_values")
            )
            .group_by("user_id")
            .agg(pl.col("sorted_values").head(k).alias("top_values"))
            .with_columns(
                pl.col("top_values").list.to_struct(fields=[f"top{i}_cat_in_{suf}" for i in range(1, k + 1)])
            )
            .unnest(columns="top_values")
        ),
        on='user_id', how='left'
    )
    
for col in [col for col in df.columns if col.find("cat") != -1]:
    df = df.with_columns(pl.col(col).fill_null(""))

  0%|          | 0/4 [00:00<?, ?it/s]

### Топ-k типов виджетов на виды активности

In [39]:
actions_id_to_suf = {
    1: "click",
    2: "favorite",
    # 3: "order", так как в заказе только один виджет, нет смысла
    5: "to_cart",
}
for id_, suf in tqdm(actions_id_to_suf.items()):
    df = df.join(
        (
            actions_history
            .filter(pl.col('timestamp').dt.date() <= val_end_date)
            .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=30 * 4))
            .filter(pl.col("action_type_id") == id_)
            .group_by("user_id", "widget_name_id")
            .agg(pl.len().alias("count"))
            .with_columns(
                pl.col("widget_name_id").sort_by(by=["count", "widget_name_id"], descending=[True, False])
                .over("user_id")
                .alias("sorted_values")
            )
            .group_by("user_id")
            .agg(pl.col("sorted_values").head(k).alias("top_values"))
            .with_columns(
                pl.col("top_values").list.to_struct(fields=[f"top{i}_widget_in_{suf}" for i in range(1, k + 1)])
            )
            .unnest(columns="top_values")
        ),
        on='user_id', how='left'
    )

df = df.join(
        (
        search_history
        .filter(pl.col('timestamp').dt.date() <= val_end_date)
        .filter(pl.col('timestamp').dt.date() >= val_end_date - timedelta(days=30 * 4))
        .filter(pl.col("action_type_id") == 4)
        .group_by("user_id", "widget_name_id")
        .agg(pl.len().alias("count"))
        .with_columns(
            pl.col("widget_name_id").sort_by(by=["count", "widget_name_id"], descending=[True, False])
            .over("user_id")
            .alias("sorted_values")
        )
        .group_by("user_id")
        .agg(pl.col("sorted_values").head(k).alias("top_values"))
        .with_columns(
            pl.col("top_values").list.to_struct(fields=[f"top{i}_widget_in_search" for i in range(1, k + 1)])
        ).unnest(columns="top_values")
    ),
    on='user_id', how='left'
)

for col in [col for col in df.columns if col.find("widget") != -1]:
    df = df.with_columns(pl.col(col).fill_null(0))

  0%|          | 0/3 [00:00<?, ?it/s]

In [40]:
df_pd = df.to_pandas()

In [41]:
df_pd.shape

(2068424, 177)

In [ ]:
df_pd['predict'] = model.predict(df_pd[cols], prediction_type="Probability")[:, 1]

In [45]:
df_pd[['user_id', 'predict']]

,user_id,predict_gbm
0,1342,0.179478
1,9852,0.740126
2,10206,0.215897
3,11317,0.216043
4,13289,0.514379
...,...,...
2068419,11157283,0.224369
2068420,11160395,0.112995
2068421,11165052,0.590553
2068422,11168218,0.554029


In [46]:
df_pd[['user_id', 'predict']].to_csv('new_try.csv', index=False)

In [47]:
df_pd.memory_usage(deep=True).sum() / 1024**2

1788.6023750305176

In [48]:
len(train_data.get_feature_name())

176